In [ ]:
import torch
import torch.nn as nn
import numpy as np
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(3, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 2)

        )
    def forward(self, x, y, t):
        inputs = torch.cat([x, y, t], dim = 1)
        u = self.hidden(inputs)
        v = self.hidden(inputs)
        #w = self.hidden(inputs)
        return u, v
    print("Hiefere")


Hiefere


In [ ]:
# define the pde residual function
def pde_residual(x, y, t, model, nu = 0.01):

    x.requires_grad = True
    y.requires_grad = True
    #z.requires_grad = True
    t.requires_grad = True

    #u = model(x, t) #predict u from the model
    #u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    #u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    #u_y = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    #u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
    #u_yy = torch.autograd.grad(u_y, y, torch.ones_like(u_y), create_graph=True)[0]
    #v_t = torch.autograd.grad(v, t, torch.ones_like(v), create_graph=True)[0]
    #v_x = torch.autograd.grad(v, x, torch.ones_like(v), create_graph=True)[0]
    #v_y = torch.autograd.grad(v, y, torch.ones_like(v), create_graph=True)[0]
    #v_xx = torch.autograd.grad(u_x, x, torch.ones_like(v_x), create_graph=True)[0]
    #v_yy = torch.autograd.grad(u_y, y, torch.ones_like(v_y), create_graph=True)[0]
    #w_t = torch.autograd.grad(w, t, torch.ones_like(u), create_graph=True)[0]
    #v_z = torch.autograd.grad(v, z, torch.ones_like(v), create_graph=True)[0]
    #w_x = torch.autograd.grad(w, x, torch.ones_like(w), create_graph=True)[0]
    #w_y = torch.autograd.grad(w, y, torch.ones_like(w), create_graph=True)[0]
    #w_z = torch.autograd.grad(w, z, torch.ones_like(w), create_graph=True)[0]
    #u_z = torch.autograd.grad(u, z, torch.ones_like(u), create_graph=True)[0]
    #u_zz = torch.autograd.grad(u_z, z, torch.ones_like(u_z), create_graph=True)[0]
    #v_zz = torch.autograd.grad(v_x, x, torch.ones_like(v_x), create_graph=True)[0]

    #residual = (u_t + u * u_x + v*u_y - nu * (u_xx + u_yy + u_zz))  +  (v_t + u * v_x + v*v_y + - nu * (v_xx + v_yy + v_zz))
    #return residual


def pde_residual(x, y, t, model, nu):

    x.requires_grad_(True)
    y.requires_grad_(True)
    t.requires_grad_(True)
    u, v = model(x, y, t)
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    f_u = u_t + u * u_x + v * u_y - nu * (u_xx + u_yy)
    f_v = v_t + u * v_x + v * v_y - nu * (v_xx + v_yy)

    return f_u, f_v

#initial and boundary condition
def initial_condition(x, y):
    u0 = -torch.sin(np.pi * x) * torch.sin(np.pi * y)
    v0 = torch.zeros_like(u0)
    return u0, v0

def boundary_condition(x, y, t):
    u_bc = torch.zeros_like(x)
    v_bc = torch.zeros_like(y)
    return u_bc, v_bc

print("2")


print(f'x shape: {x.shape}')
print(f'y shape: {y.shape}')
print(f't shape: {t.shape}')


2
x shape: torch.Size([200, 1])
y shape: torch.Size([200, 1])
t shape: torch.Size([100, 1])


In [ ]:
# train the model
x = torch.linspace(-1, 1, 200).view(-1, 1)
y = torch.linspace(-1, 1, 200).view(-1, 1)
#z = torch.linspace(-1, 1, 200).view(-1, 1)
t = torch.linspace(0, 1, 100).view(-1, 1)
x_train, y_train, t_train = torch.meshgrid(x.squeeze(), y.squeeze(), t.squeeze(), indexing = 'ij')
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
t_train = t_train.reshape(-1, 1)
print("hi")

hi


In [ ]:
#define the model, optimizer, and loss function
import torch.optim as optim
import torch.nn as nn
loss_fn = nn.MSELoss()


model = PINN()
optimizer = optim.Adam(model.parameters(), lr = 0.01)


In [ ]:
def loss_function(model, x, y, t, nu):
    # Initial condition loss
    u_pred, v_pred = model(x, y, torch.zeros_like(t))
    u0, v0 = initial_condition(x, y)
    loss_ic = torch.mean((u_pred - u0)**2 + (v_pred - v0)**2)

    # Boundary condition loss (assuming boundaries at x=±1, y=±1)
    u_pred_left, v_pred_left = model(torch.full_like(t, -1), y, t)
    u_pred_right, v_pred_right = model(torch.full_like(t, 1), y, t)
    u_pred_bottom, v_pred_bottom = model(x, torch.full_like(t, -1), t)
    u_pred_top, v_pred_top = model(x, torch.full_like(t, 1), t)
    u_bc, v_bc = boundary_condition(x, y, t)
    loss_bc = torch.mean((u_pred_left - u_bc)**2 + (u_pred_right - u_bc)**2 +
                         (u_pred_bottom - u_bc)**2 + (u_pred_top - u_bc)**2 +
                         (v_pred_left - v_bc)**2 + (v_pred_right - v_bc)**2 +
                         (v_pred_bottom - v_bc)**2 + (v_pred_top - v_bc)**2)

    # PDE residual loss
    f_u, f_v = pde_residual(x, y, t, model, nu)
    loss_pde = torch.mean(f_u**2 + f_v**2)

    loss = loss_ic + loss_bc + loss_pde
    return loss


In [ ]:
num_epochs = 12000
print("Hi")
for each in range(num_epochs):
    model.train()

    # Compute initial condition loss
    #u_pred = model(x, y, torch.zeros_like(x))
    #u_true = initial_condition(x, y)
    #v_pred = model(x, y, torch.zeros_like(y))
    #v_true = initial_condition(x, y)
    #loss_ic = torch.mean((u_pred - u_true)**2 + (v_pred - v_true)**2)m

    # Compute boundary condition loss
    #u_pred_left = model(torch.full_like(t, -1), t) # u(-1, t)
    #u_pred_right = model(torch.full_like(t, 1), t) # u(1, t)
    #v_pred_left = model(torch.full_like(t, -1), t) # u(-1, t)
    #v_pred_right = model(torch.full_like(t, 1), t) # u(1, t)
    #loss_bc = torch.mean((u_pred_left - boundary_condition(torch.full_like(t, -1), t))**2) + \
              #torch.mean((u_pred_right - boundary_condition(torch.full_like(t, 1), t))**2) + torch.mean((u_pred_left - boundary_condition(torch.full_like(t, -1), t))**2) + \
              #torch.mean((u_pred_right - boundary_condition(torch.full_like(t, 1), t))**2)

    # Compute PDE residual loss
    #residual = pde_residual(x_train, y_train, t_train, model)
    #loss_pde = torch.mean(residual**2)

    #loss = loss_ic + loss_bc + loss_pde

    # Initial condition loss
    u_pred, v_pred = model(x, y, torch.zeros_like(t))
    u0, v0 = initial_condition(x, y)
    loss_ic = torch.mean((u_pred - u0)**2 + (v_pred - v0)**2)

    # Boundary condition loss (assuming boundaries at x=±1, y=±1)
    u_pred_left, v_pred_left = model(torch.full_like(t, -1), y, t)
    u_pred_right, v_pred_right = model(torch.full_like(t, 1), y, t)
    u_pred_bottom, v_pred_bottom = model(x, torch.full_like(t, -1), t)
    u_pred_top, v_pred_top = model(x, torch.full_like(t, 1), t)
    u_bc, v_bc = boundary_condition(x, y, t)
    loss_bc = torch.mean((u_pred_left - u_bc)**2 + (u_pred_right - u_bc)**2 +
                         (u_pred_bottom - u_bc)**2 + (u_pred_top - u_bc)**2 +
                         (v_pred_left - v_bc)**2 + (v_pred_right - v_bc)**2 +
                         (v_pred_bottom - v_bc)**2 + (v_pred_top - v_bc)**2)

    # PDE residual loss
    f_u, f_v = pde_residual(x, y, t, model, nu)
    loss_pde = torch.mean(f_u**2 + f_v**2)

    loss = loss_ic + loss_bc + loss_pde

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     print("Hi")

    if each % 500 == 0:
        print(f'Epoch [{each + 1}/{num_epochs}], Loss: {loss.item():.4f}')




Hi


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 200 but got size 100 for tensor number 2 in the list.

In [ ]:
# evaluate the model on a grid of (x, t) values
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
x_test = torch.linspace(-1, 1, 100).view(-1, 1)
t_test = torch.linspace(0, 1, 100).view(-1, 1)
x_test, t_test  = torch.meshgrid(x_test.squeeze(), t_test.squeeze(), indexing = 'xy')
x_test = x_test.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)

model.eval()
with torch.no_grad():
    u_pred = model(x_test, t_test).numpy()

# reshape the predicted u values for contour plotting
x_test = x_test.numpy().reshape(100, 100)
t_test = t_test.numpy().reshape(100, 100)
u_pred = u_pred.reshape(100, 100)

#plot the PINN solution as a contour plot
plt.figure(figsize = (16, 4))
plt.contourf(x_test, t_test, u_pred, levels = 250, cmap = 'jet')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('t')
plt.title('solution of burgers\' equation using PINNs')
plt.show()

#%%

#load the reference solution data
data= np.load("Burgers.npz")
t_ref, x_ref, exact = data["t"], data["x"], data["usol"].T
#Reshape
x_ref, t_ref = np.meshgrid(x_ref, t_ref)

# Define spatial and temporal grids
##x_ref = np.linspace(-1, 1, 100)  # 100 spatial points
#t_ref = np.linspace(0, 1, 100)  # 100 time points

# Example: Compute an analytical solution for visualization
#exact = np.sin(np.pi * x_ref[None, :]) * np.exp(-t_ref[:, None])

# Plot the reference solution as a contour plot
plt.figure(figsize=(16, 4))
plt.contourf(x_ref, t_ref, exact, levels=250, cmap='jet')
plt.colorbar()
plt.xlabel('x')
plt.ylabel('t')
plt.title('Reference Solution of burgers\' equation')
plt.show()



TypeError: PINN.forward() missing 1 required positional argument: 't'

In [ ]:

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 6))

contour1 = ax1.contourf(x_test, t_test, u_pred, levels = 250, cmap = 'jet')
fig.colorbar(contour1, ax = ax1)
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_title('solution of burger\' equation uaing pinn (cpu)')


#plot reference
contour2 = ax2.contourf(x_ref, t_ref, exact, levels = 250, cmap = 'jet')
fig.colorbar(contour2, ax = ax2)
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_title('refernce solution of burger\' equation uaing pinn (cpu)')
#set same limits for both plots
ax1.set_xlim([x_ref.min(), x_ref.max()])
ax1.set_ylim([t_ref.min(), t_ref.max()])
ax2.set_xlim([x_ref.min(), x_ref.max()])
ax2.set_ylim([t_ref.min(), t_ref.max()])

plt.tight_layout()
plt.show()







In [ ]:

import matplotlib.pyplot as plt

time_slices = [ 0.25, 0.5, 0.75]
num_plots= len(time_slices)
cols = 2
rows = (num_plots // cols) +(num_plots % cols)
fig, axs = plt.subplots(rows, cols, figsize = (12, 5*rows))
axs = axs.flatten()
for i, t_val in enumerate(time_slices):
    # Find closest time point in t_ref
    idx_ref = np.argmin(np.abs(t_ref[:,0] - t_val))
    u_ref_slice = exact[idx_ref,:]

    t_slice = t_val * np.ones((100, 1))
    x_slice = np.linspace(-1, 1, 100).reshape(-1, 1)

    model.eval()
    with torch.no_grad():
        u_pinn_slice = model(torch.tensor(x_slice, dtype=torch.float32),
                             torch.tensor(t_slice, dtype=torch.float32)).numpy()

    axs[i].plot(x_ref[0, :], u_ref_slice, 'r-', label='Reference Solution')
    axs[i].plot(x_slice, u_pinn_slice, 'b--', label='PINN Solution')

    axs[i].set_xlabel('x')
    axs[i].set_ylabel('u')
    axs[i].set_title(f'solution at t = {t_val}')
    axs[i].legend()

for j in range(num_plots, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()